In [1]:
import requests
import csv
import numpy as np
from datetime import datetime
import time

monedas = ['bitcoin', 'ethereum', 'binancecoin', 'cardano', 'dogecoin', 'polkadot', 'ripple', 'solana', 'tether', 'matic-network']
URL_BASE = "https://api.coingecko.com/api/v3/coins/"

# Convertir la fecha "2020/09/18" a un objeto datetime
fecha_especifica = datetime.strptime("2020/08/19", "%Y/%m/%d")

# Convertir el objeto datetime a una marca de tiempo Unix
fecha_para_19_8_20 = time.mktime(fecha_especifica.timetuple())

# Obtener marca de tiempo Unix actual
FECHA_ACTUAL = int(time.time())

def obtener_datos(moneda, desde_FECHA, hasta_FECHA):
    url = f"{URL_BASE}{moneda}/market_chart/range?vs_currency=usd&from={fecha_para_19_8_20}&to={FECHA_ACTUAL}"
    respuesta = requests.get(url)
    datos = respuesta.json()
    return datos

def calcular_volatilidad_30_dias(precios):
    # Calcular rendimientos diarios
    rendimientos = [((precios[i] - precios[i - 1]) / precios[i - 1]) for i in range(1, len(precios))]
    
    # Calcular la volatilidad (desviación estándar de los rendimientos)
    volatilidad = np.std(rendimientos) * np.sqrt(30)  # Asumiendo 30 días en un mes
    return volatilidad

def guardar_en_csv(moneda, datos):
    with open(f"{moneda}.csv", "w", newline='') as archivo:
        # Escribir los encabezados primero
        archivo.write("Fecha,Precio,MarketCap,Volumen24h,Volatilidad30Dias\n")

        precios = [item[1] for item in datos['prices']]
        
        for i in range(len(datos['prices'])):
            fecha = datetime.utcfromtimestamp(datos['prices'][i][0] / 1000).strftime('%Y-%m-%d')
            precio = "{:.3f}".format(datos['prices'][i][1])
            
            # Comprobar si el índice i existe en 'market_caps' y 'total_volumes'
            market_cap = "{:.3f}".format(datos['market_caps'][i][1]) if i < len(datos['market_caps']) else 0
            volumen = "{:.3f}".format(datos['total_volumes'][i][1]) if i < len(datos['total_volumes']) else 0
            
            # Calcular la volatilidad de los últimos 30 días si hay suficientes datos disponibles
            if i >= 29:
                volatilidad = "{:.3f}".format(calcular_volatilidad_30_dias(precios[i-29:i+1]))
            else:
                volatilidad = float(0)
            
            fila = f"{fecha},{precio},{market_cap},{volumen},{volatilidad}"
            
            # Solo añade un salto de línea si no es el último elemento
            archivo.write(fila + ("\n" if i < len(datos['prices']) - 1 else ""))



def borrar_ultimas_filas_vacias_o_incompletas(csv_file_path):
    with open(csv_file_path, 'r', newline='') as file:
        lineas = file.readlines()
       
        # Eliminar la última fila
        lineas = lineas[:-1]

        # Quitar el salto de línea del final para evitar un salto extra al escribir el archivo
        lineas[-1] = lineas[-1].rstrip('\n')

    with open(csv_file_path, 'w', newline='') as file:
        file.writelines(lineas)


for moneda in monedas:
        datos = obtener_datos(moneda, fecha_para_19_8_20, FECHA_ACTUAL)
        guardar_en_csv(moneda, datos)
        borrar_ultimas_filas_vacias_o_incompletas(f"{moneda}.csv")
